In [109]:
import sys
sys.path.append('..')

In [110]:
import numpy

In [111]:
# p = numpy.array(
#     [
#         [11, 12, 13],
#         [21, 22, 23],
#         [31, 32, 33],
#     ]
# )


p = numpy.array(
    [
        [11, 12, 13, 14, 15],
        [21, 22, 23, 24, 25],
        [31, 32, 33, 34, 35],
        [41, 42, 43, 44, 45],
        [51, 52, 53, 54, 55],
    ]
)
p

array([[11, 12, 13, 14, 15],
       [21, 22, 23, 24, 25],
       [31, 32, 33, 34, 35],
       [41, 42, 43, 44, 45],
       [51, 52, 53, 54, 55]])

In [112]:
import lasp.utils
# p_shifted = lasp.utils.pad_circshift_center(p, p.shape)
center = numpy.array(p.shape) // 2
p_shifted = numpy.roll(p, -center, [0, 1])
print(p_shifted)

[[33 34 35 31 32]
 [43 44 45 41 42]
 [53 54 55 51 52]
 [13 14 15 11 12]
 [23 24 25 21 22]]


In [113]:
def make_bccb(p: numpy.ndarray) -> numpy.ndarray :
    
    N, M = p.shape
    
    dim = numpy.prod(numpy.array(p.shape))
    p_cpy = numpy.copy(p)
    
    p1 = numpy.reshape(p_cpy.T, newshape=dim)
    P = numpy.zeros(shape=(dim, dim))
    P[:, 0] = p1

    # print("First Boucle")
    # print(P)
    for j in range(1, M):
        for i in range(0, M*N, N):
            P[i:i+N, j] = numpy.roll(P[i:i+N, j-1], 1)
            # print(P)

    # print("Second Boucle")
    for j in range(1, M):
        P[:, N*j: N*j+N] = numpy.roll(P[:, N*(j-1): N*(j-1)+N], (N, 0), (0, 1))
        # print(P)


    return P

In [114]:
bccb = make_bccb(p_shifted)
print(bccb)


[[33. 23. 13. 53. 43. 32. 22. 12. 52. 42. 31. 21. 11. 51. 41. 35. 25. 15.
  55. 45. 34. 24. 14. 54. 44.]
 [43. 33. 23. 13. 53. 42. 32. 22. 12. 52. 41. 31. 21. 11. 51. 45. 35. 25.
  15. 55. 44. 34. 24. 14. 54.]
 [53. 43. 33. 23. 13. 52. 42. 32. 22. 12. 51. 41. 31. 21. 11. 55. 45. 35.
  25. 15. 54. 44. 34. 24. 14.]
 [13. 53. 43. 33. 23. 12. 52. 42. 32. 22. 11. 51. 41. 31. 21. 15. 55. 45.
  35. 25. 14. 54. 44. 34. 24.]
 [23. 13. 53. 43. 33. 22. 12. 52. 42. 32. 21. 11. 51. 41. 31. 25. 15. 55.
  45. 35. 24. 14. 54. 44. 34.]
 [34. 24. 14. 54. 44. 33. 23. 13. 53. 43. 32. 22. 12. 52. 42. 31. 21. 11.
  51. 41. 35. 25. 15. 55. 45.]
 [44. 34. 24. 14. 54. 43. 33. 23. 13. 53. 42. 32. 22. 12. 52. 41. 31. 21.
  11. 51. 45. 35. 25. 15. 55.]
 [54. 44. 34. 24. 14. 53. 43. 33. 23. 13. 52. 42. 32. 22. 12. 51. 41. 31.
  21. 11. 55. 45. 35. 25. 15.]
 [14. 54. 44. 34. 24. 13. 53. 43. 33. 23. 12. 52. 42. 32. 22. 11. 51. 41.
  31. 21. 15. 55. 45. 35. 25.]
 [24. 14. 54. 44. 34. 23. 13. 53. 43. 33. 22. 12. 52. 4

In [115]:
bccb_col = bccb[:, 0]
reshaped = bccb_col.reshape(p.shape)
# print(reshaped)
print(numpy.roll(reshaped, center, [0, 1]))

[[11. 21. 31. 41. 51.]
 [12. 22. 32. 42. 52.]
 [13. 23. 33. 43. 53.]
 [14. 24. 34. 44. 54.]
 [15. 25. 35. 45. 55.]]


In [116]:
bccb_col = bccb.T[:, 0]
reshaped = bccb_col.reshape(p.shape)
# print(reshaped)
print(numpy.roll(reshaped, center, [0, 1]))

[[55. 45. 35. 25. 15.]
 [54. 44. 34. 24. 14.]
 [53. 43. 33. 23. 13.]
 [52. 42. 32. 22. 12.]
 [51. 41. 31. 21. 11.]]


In [117]:
numpy.flip(p)

array([[55, 54, 53, 52, 51],
       [45, 44, 43, 42, 41],
       [35, 34, 33, 32, 31],
       [25, 24, 23, 22, 21],
       [15, 14, 13, 12, 11]])